In [14]:
# Dataset
from sklearn import datasets

# Data processing
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Model and performace evaluation
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support as score

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, space_eval

%reload_ext watermark
%watermark -iv -v

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.21.0

sklearn : 1.4.1.post1
pandas  : 2.2.0
numpy   : 1.26.4
hyperopt: 0.2.7



In [15]:
# Load the breast cancer dataset
data = datasets.load_breast_cancer()

# Put the data in pandas dataframe format
df = pd.DataFrame(data=data.data, columns=data.feature_names)
df['target'] = data.target

# Check the data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [16]:
# Check the target value distribution
df['target'].value_counts(normalize=True)

target
1    0.627417
0    0.372583
Name: proportion, dtype: float64

In [17]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(df[df.columns.difference(['target'])],
                                                    df['target'],
                                                    test_size=.2,
                                                    random_state=42)

# Check the number of records in training and testing dataset
print(f'The training dataset has {len(X_train)} records.')
print(f'The training dataset has {len(X_test)} records.')

The training dataset has 455 records.
The training dataset has 114 records.


In [18]:
# Initiate scaler
sc = StandardScaler()

# StandardScaler the training dataset
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train), index=X_train.index, columns=X_train.columns)

# StandardScaler the testing dataset
X_test_transformed = pd.DataFrame(sc.fit_transform(X_test), index=X_test.index, columns=X_test.columns)

# Summary statistics after standardization
X_train_transformed.describe().T

,count,mean,std,min,25%,50%,75%,max
area error,455.0,6.246530e-17,1.001101,-0.705091,-0.464164,-0.325347,0.077435,10.641841
compactness error,455.0,-2.395154e-15,1.001101,-1.258102,-0.694353,-0.280607,0.358304,5.905671
concave points error,455.0,3.455112e-16,1.001101,-1.891775,-0.668493,-0.126279,0.437566,6.504667
concavity error,455.0,2.479091e-16,1.001101,-1.022218,-0.551340,-0.207836,0.303371,11.310294
fractal dimension error,455.0,5.085065e-16,1.001101,-1.050856,-0.573964,-0.218908,0.245340,9.345870
mean area,455.0,-2.537653e-16,1.001101,-1.365036,-0.660205,-0.289597,0.319339,5.208312
mean compactness,455.0,1.011157e-15,1.001101,-1.607228,-0.777087,-0.241340,0.528128,3.964311
mean concave points,455.0,5.817081e-16,1.001101,-1.269910,-0.734905,-0.391123,0.673757,4.022271
mean concavity,455.0,9.857804e-16,1.001101,-1.119899,-0.750539,-0.344646,0.547387,4.256736
mean fractal dimension,455.0,-3.367270e-15,1.001101,-1.776889,-0.709792,-0.177285,0.464223,4.815921


In [19]:
# Summary statistics before standardization
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
area error,455.0,40.071299,47.236319,6.802000,18.170000,24.720000,43.725000,542.20000
compactness error,455.0,0.025635,0.018607,0.002252,0.012730,0.020420,0.032295,0.13540
concave points error,455.0,0.011894,0.006294,0.000000,0.007691,0.011100,0.014645,0.05279
concavity error,455.0,0.032824,0.032146,0.000000,0.015120,0.026150,0.042565,0.39600
fractal dimension error,455.0,0.003820,0.002787,0.000895,0.002222,0.003211,0.004504,0.02984
mean area,455.0,654.377582,354.943187,170.400000,420.300000,551.700000,767.600000,2501.00000
mean compactness,455.0,0.103619,0.052470,0.019380,0.062890,0.090970,0.131300,0.31140
mean concave points,455.0,0.048280,0.038060,0.000000,0.020340,0.033410,0.073895,0.20120
mean concavity,455.0,0.088898,0.079468,0.000000,0.029320,0.061540,0.132350,0.42680
mean fractal dimension,455.0,0.062757,0.007210,0.049960,0.057645,0.061480,0.066100,0.09744


In [20]:
# Initiate XGBoost Classifier
xgboost = XGBClassifier()

# Print default setting
xgboost.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [21]:
# Train the model
xgboost = XGBClassifier(seed=0).fit(X_train_transformed, y_train)

# Make prediction
xgboost_predict = xgboost.predict(X_test_transformed)

# Get predicted probability
xgboost_predict_prob = xgboost.predict_proba(X_test_transformed)[:,1]

In [22]:
# Get performance metrics
precision, recall, fscore, support = score(y_test, xgboost_predict)

# Print result
print(f'The recall value for the baseline xgboost model is {recall[1]:.2%}')

The recall value for the baseline xgboost model is 98.59%


In [23]:
# Define the search space
param_grid = {
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [ 0.3, 0.5, 0.8 ],
    # reg_alpha provides 11 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [ 0, 0.5, 1, 5 ],
    # reg_lambda provides 12 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [ 0, 0.5, 1, 5]
}

# Set up score
scoring = ['recall']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [24]:
# Define grid search
grid_search = GridSearchCV(estimator=xgboost,
                           param_grid=param_grid,
                           scoring=scoring,
                           refit='recall',
                           n_jobs=-1,
                           cv=kfold,
                           verbose=0)

# Fit grid search
grid_result = grid_search.fit(X_train_transformed, y_train)

# Print grid search summary
grid_result

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.5, 0.8],
                         'reg_alpha': [0, 0.5, 1, 5],
                         'reg_lambda': [0, 0.5, 1, 5]},
             refit='recall', scoring=['recall'])

In [25]:
# Print the best score and the corresponding hyperparameters
print(f'The best score is {grid_result.best_score_:.4f}')
print(f'The best score standard deviation is {grid_result.cv_results_['std_test_recall'][grid_result.best_index_]:.4f}')
print(f'The best hyperparameters are {grid_result.best_params_}')

The best score is 0.9860
The best score standard deviation is 0.0050
The best hyperparameters are {'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 0}


In [26]:
# Make prediction using the best model
grid_predict = grid_search.predict(X_test_transformed)

# Get predicted probabilities
grid_predict_prob = grid_search.predict_proba(X_test_transformed)[:,1]

# Get performance metrics
precision, recall, fscore, support = score(y_test, grid_predict)

# Print result
print(f'The recall value for the xgboost grid search is {recall[1]:.2%}')

The recall value for the xgboost grid search is 98.59%


In [27]:
# Define the search space
param_grid = {
    # Learning rate shrinks the weights to make the boosting process more conservative
    "learning_rate": [0.0001, 0.001, 0.01, 0.1, 1],
    # Maximum depth of the tree, increasing it increases the model complexity.
    "max_depth": range(3, 21, 3),
    # Gamma specifies the minumum loss reduction required to make a split.
    "gamma": [i/10.0 for i in range(0,5)],
    # Percentage of columns to be randomly samples for each tree.
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    # reg_alpha provides 11 regularization to the weight, higher values result in more conservative models
    "reg_alpha": [ 1e-5, 1e-2, 0.1, 1, 10, 100 ],
    # reg_lambda provides 12 regularization to the weight, higher values result in more conservative models
    "reg_lambda": [ 1e-5, 1e-2, 0.1, 1, 10, 100 ]}

# Set up score
scoring = ['recall']

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [28]:
# Define random search
random_search = RandomizedSearchCV(estimator=xgboost,
                                   param_distributions=param_grid,
                                   n_iter=48,
                                   scoring=scoring,
                                   refit='recall',
                                   n_jobs=-1,
                                   cv=kfold,
                                   verbose=0)

# Fit grid search
random_result = random_search.fit(X_train_transformed, y_train)

# Print grid search summary
random_result

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importanc...
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=48, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                                             0.7, 0.8, 0.9],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1],
                                        'max_depth': range(3, 21, 3),
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 10,
                                                      100],
                                        'reg_lambda': [1e-05, 0.01, 0.1, 1, 10,
                                                       100]},
                   refit='recall', scoring=['recall'])

In [29]:
# Print the best score and the corresponding hyperparameters
print(f'The best score is {random_result.best_score_:.4f}')
print(f'The best score standard deviation is {random_result.cv_results_['std_test_recall'][random_result.best_index_]:.2f}')
print(f'The best hyperparameters are {random_result.best_params_}')

The best score is 1.0000
The best score standard deviation is 0.00
The best hyperparameters are {'reg_lambda': 0.1, 'reg_alpha': 10, 'max_depth': 12, 'learning_rate': 0.001, 'gamma': 0.2, 'colsample_bytree': 0.5}


In [30]:
# Make prediction using the best model
random_predict = random_search.predict(X_test_transformed)

# Get predicted probabilities
random_predict_prob = random_search.predict_proba(X_test_transformed)[:,1]

# Get performance metrics
precision, recall, fscore, support = score(y_test, random_predict)

# Print result
print(f'The recall value for the xgboost grid search is {recall[1]:.2%}')

The recall value for the xgboost grid search is 100.00%


c:\Program Files\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
